In [28]:
# module import
import numpy as np
import re
import pickle
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [20]:
# 경로 설정
root_path = "/content/drive/My Drive/멀티캠퍼스/[혁신성장] 인공지능 자연어처리 기반/[강의]/조성현 강사님"
data_path = f"{root_path}/dataset"

# _1_. SVD

 특이값 분해

* U : 특이값 가진 행렬.
* S : 앞 부분 r개만 선택해서 자른 행렬. 대각 원소만 나오므로 행렬 변환해야 함.
* D : 분해할 행렬의 주요 성분을 가지고 있는 행렬.

In [73]:
statements = [
              'ruled India',
              'Chalukyas ruled Badami',
              'So many kingdoms ruled India',
              'Lalbagh is a botanical garden in India'
              ]

# Tf-idf 행렬 생성
tf_vector = TfidfVectorizer(max_features=8) # 빈도 수 높은 8개 단어만 선택
tfidf = tf_vector.fit_transform(statements) # tfidf 행렬 생성
print(type(tfidf), tfidf.shape) # sparse matrix
print(tfidf.toarray(), '\n') # array 형태로 변환하여 확인

# SVD로 Tf-idf 행렬 분해
U, S, VT = np.linalg.svd(tfidf.toarray(), full_matrices=True)
print(f"========= U ========= :\n{U.round(2)}\]\n")
print(f"========= S ========= :\n{S.round(2)}\n")
print(f"========= VT ========= :\n{VT.round(2)}\n")

# S 행렬 형태로 변환
S_mat = np.zeros(tfidf.shape)
S_mat[:S.shape[0], :S.shape[0]] = np.diag(S)
print(f"========= S Matrix ========= :\n{S_mat.round(2)}\n")

<class 'scipy.sparse.csr.csr_matrix'> (4, 8)
[[0.         0.         0.         0.         0.         0.70710678
  0.         0.70710678]
 [0.64450299 0.         0.64450299 0.         0.         0.
  0.         0.41137791]
 [0.         0.         0.         0.         0.         0.70710678
  0.         0.70710678]
 [0.         0.47633035 0.         0.47633035 0.47633035 0.30403549
  0.47633035 0.        ]] 

========= U ========= :
[[-0.65  0.    0.27 -0.71]
 [-0.31 -0.59 -0.74 -0.  ]
 [-0.65 -0.    0.27  0.71]
 [-0.23  0.8  -0.55  0.  ]]\]

========= S ========= :
[1.49 1.   0.89 0.  ]

========= VT ========= :
[[-0.14 -0.07 -0.14 -0.07 -0.07 -0.67 -0.07 -0.71]
 [-0.38  0.38 -0.38  0.38  0.38  0.24  0.38 -0.24]
 [-0.54 -0.29 -0.54 -0.29 -0.29  0.25 -0.29  0.09]
 [ 0.74 -0.03 -0.62 -0.03 -0.03  0.19 -0.03 -0.19]
 [-0.   -0.16 -0.03 -0.49  0.84 -0.04 -0.16  0.04]
 [ 0.    0.66  0.2  -0.52 -0.17  0.32 -0.17 -0.32]
 [-0.   -0.16 -0.03 -0.49 -0.16 -0.04  0.84  0.04]
 [ 0.    0.53 -0.35 -0.

# _2_. LSA

- 뉴스 데이터 활용: 다운로드 시간 오래 걸리므로 강사님이 미리 받아 놓음.
- `scikit-learn`의 `TruncatedSVD` 활용: 위와 사용법 다름.


In [56]:
# 데이터 로드
with open(f"{data_path}/news.data", 'rb') as f:
    news_data = pickle.load(f)

In [57]:
# 데이터 조회
news = news_data.data
print(len(news)) # 총 11314개의 뉴스 데이터
print('')
print(news[0])
print('')

# target 확인: 뉴스별로 분류되어 있는 topic
print(news_data.target_names)
print(len(news_data.target_names)) # 주제 총 20개

11314

Well i'm not sure about the story nad it did seem biased. What
I disagree with is your statement that the U.S. Media is out to
ruin Israels reputation. That is rediculous. The U.S. media is
the most pro-israeli media in the world. Having lived in Europe
I realize that incidences such as the one described in the
letter have occured. The U.S. media as a whole seem to try to
ignore them. The U.S. is subsidizing Israels existance and the
Europeans are not (at least not to the same degree). So I think
that might be a reason they report more clearly on the
atrocities.
	What is a shame is that in Austria, daily reports of
the inhuman acts commited by Israeli soldiers and the blessing
received from the Government makes some of the Holocaust guilt
go away. After all, look how the Jews are treating other races
when they got power. It is unfortunate.


['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.fo

In [58]:
# 전처리
news_1 = []
news_2 = []

# 1) 영문자 아닌 문자 모두 제거
for doc in news:
    news_1.append(re.sub("[^a-zA-Z]", " ", doc))

# 2) 소문자 변환, 불용어 제거, 3글자 이상인 단어만 사용.
stopwords_list = stopwords.words('english') # 영어 불용어 리스트
for doc in news_1:
    doc_temp = []
    for w in doc.split():
        w = w.lower() # 소문자 변환
        if len(w) > 3 and w not in stopwords_list: # 조건에 맞는 단어들만 선택
            doc_temp.append(w)
    news_2.append(" ".join(doc_temp)) # 문자열로 바꾸어서 선택

print(news_2[0]) # 위의 원문과 비교해 어떻게 달라졌는지 확인

well sure story seem biased disagree statement media ruin israels reputation rediculous media israeli media world lived europe realize incidences described letter occured media whole seem ignore subsidizing israels existance europeans least degree think might reason report clearly atrocities shame austria daily reports inhuman acts commited israeli soldiers blessing received government makes holocaust guilt away look jews treating races power unfortunate


In [59]:
# Tf-idf matrix
tf_vector = TfidfVectorizer(max_features=500) # 빈도가 높은 500개 단어만 선택
tf_vector.fit(news_2) # fit하기
print(tf_vector.vocabulary_) # 어휘 목록 확인: 500개.
tfidf_matrix = tf_vector.transform(news_2)
print(tfidf_matrix.shape)

# 어휘 목록 확인 시 이렇게 해도 됨.
vocab = tf_vector.get_feature_names()
print(vocab[476]) # 476번째 확인하면 위와 똑같음.

{'well': 476, 'sure': 420, 'seem': 370, 'statement': 410, 'israeli': 212, 'world': 493, 'whole': 481, 'least': 229, 'think': 435, 'might': 263, 'reason': 346, 'government': 170, 'makes': 253, 'away': 34, 'look': 244, 'jews': 216, 'power': 322, 'people': 303, 'read': 342, 'actually': 2, 'hard': 180, 'need': 278, 'little': 240, 'faith': 142, 'runs': 358, 'sorry': 397, 'ever': 130, 'anyway': 21, 'maybe': 258, 'start': 407, 'much': 272, 'although': 10, 'points': 315, 'would': 494, 'still': 413, 'like': 235, 'know': 222, 'question': 337, 'sort': 398, 'want': 471, 'israel': 211, 'must': 273, 'stop': 414, 'questions': 338, 'work': 490, 'last': 225, 'several': 380, 'everyone': 132, 'group': 174, 'center': 61, 'research': 351, 'nothing': 286, 'name': 274, 'change': 64, 'area': 25, 'clipper': 75, 'came': 54, 'thus': 440, 'seems': 371, 'rather': 341, 'technology': 428, 'drive': 113, 'cost': 90, 'going': 168, 'especially': 128, 'probably': 329, 'something': 394, 'real': 344, 'year': 498, 'privacy'

* `TruncatedSVD` 통해 topic 20개 feature만 선택


In [60]:
# SVD 진행
svd = TruncatedSVD(n_components=len(news_data.target_names), n_iter=1000)
svd.fit(tfidf_matrix) # 위에서 생성한 tfidf 행렬에 맞추어 특잇값분해

U = svd.fit_transform(tfidf_matrix) / svd.singular_values_
VT = svd.components_
S = np.diag(svd.singular_values_)

In [61]:
# U: 토픽 번호
print(U) # 가장 큰 값이 가장 연관성 높은 topic
print(U.shape)

[[ 0.00834054 -0.01091546 -0.00328674 ...  0.01936484 -0.01445549
   0.0079041 ]
 [ 0.0087589  -0.00410384 -0.00516331 ... -0.02103608 -0.00802416
  -0.00894954]
 [ 0.01411892 -0.00762824  0.01663664 ...  0.0210131  -0.01613651
   0.01971222]
 ...
 [ 0.00524679 -0.00311071  0.00175188 ...  0.00132466 -0.0033033
  -0.00131575]
 [ 0.00580616  0.01161377 -0.01295616 ... -0.00440827  0.0273712
  -0.02715296]
 [ 0.01586616 -0.009819   -0.01159506 ...  0.00658097 -0.00549311
   0.01398965]]
(11314, 20)


In [62]:
# 문서별 Topic 번호 확인: U에서 가장 큰 feature 인텍스 찾기
for i in range(15): # 문서 15개만 확인
    print('문서 = {:d} : Topic = {:d}'.format(i, np.argmax(U[i:(i+1), :][0]))) 

문서 = 0 : Topic = 17
문서 = 1 : Topic = 0
문서 = 2 : Topic = 17
문서 = 3 : Topic = 5
문서 = 4 : Topic = 3
문서 = 5 : Topic = 8
문서 = 6 : Topic = 0
문서 = 7 : Topic = 6
문서 = 8 : Topic = 7
문서 = 9 : Topic = 12
문서 = 10 : Topic = 1
문서 = 11 : Topic = 10
문서 = 12 : Topic = 9
문서 = 13 : Topic = 0
문서 = 14 : Topic = 6


In [63]:
# V: 토픽별 중요 단어
print(VT) # 500개의 단어 중 각 토픽별로 등장하는 중요 단어 비중
print(VT.shape)

[[ 4.55707965e-02  2.64628493e-02  5.36481364e-02 ...  4.47155421e-02
   7.38468906e-02  7.17739949e-02]
 [ 1.35724772e-02  3.66328106e-02 -2.91410583e-02 ... -2.49454075e-02
  -6.88385714e-02 -5.80195095e-02]
 [-1.29499109e-02 -2.82157474e-02 -2.21213084e-02 ... -6.87342986e-03
  -9.32670218e-02 -3.75030696e-02]
 ...
 [-1.28278335e-02 -9.90293809e-03 -3.04429513e-02 ...  1.88829364e-05
  -1.02939532e-02  3.94068164e-02]
 [-2.05348535e-02 -1.07326934e-02 -3.07443513e-03 ...  1.07910233e-02
  -2.88018613e-03 -3.63476434e-03]
 [ 5.88746214e-03  1.53334692e-02 -1.07437023e-02 ...  2.21347084e-02
   2.02575468e-01  4.00411511e-03]]
(20, 500)


In [69]:
# 토픽별 중요 단어 표시
for i in range(len(VT)):
    idx = np.flipud(VT[i].argsort())[:10] # idx 목록: 자주 등장하는 거 sparse로
    print('토픽 = {:d} {:s}'.format(i+1, news_data.target_names[i]), end=' || ') # i번째 토픽에서
    for n in idx: # idx별로 뽑아서 보자
        print('{:d}번째 단어 {:s}'.format(n, vocab[n]), end='/ ')
    print('')

토픽 = 1 alt.atheism || 494번째 단어 would/ 235번째 단어 like/ 222번째 단어 know/ 303번째 단어 people/ 435번째 단어 think/ 169번째 단어 good/ 9번째 단어 also/ 91번째 단어 could/ 441번째 단어 time/ 476번째 단어 well/ 
토픽 = 2 comp.graphics || 432번째 단어 thanks/ 484번째 단어 windows/ 312번째 단어 please/ 19번째 단어 anyone/ 250번째 단어 mail/ 56번째 단어 card/ 222번째 단어 know/ 4번째 단어 advance/ 113번째 단어 drive/ 145번째 단어 file/ 
토픽 = 3 comp.os.ms-windows.misc || 494번째 단어 would/ 432번째 단어 thanks/ 19번째 단어 anyone/ 222번째 단어 know/ 235번째 단어 like/ 312번째 단어 please/ 91번째 단어 could/ 250번째 단어 mail/ 393번째 단어 someone/ 4번째 단어 advance/ 
토픽 = 4 comp.sys.ibm.pc.hardware || 160번째 단어 game/ 427번째 단어 team/ 498번째 단어 year/ 161번째 단어 games/ 169번째 단어 good/ 225번째 단어 last/ 366번째 단어 season/ 311번째 단어 players/ 310번째 단어 play/ 190번째 단어 hockey/ 
토픽 = 5 comp.sys.mac.hardware || 494번째 단어 would/ 235번째 단어 like/ 113번째 단어 drive/ 421번째 단어 system/ 484번째 단어 windows/ 56번째 단어 card/ 365번째 단어 scsi/ 110번째 단어 disk/ 427번째 단어 team/ 330번째 단어 problem/ 
토픽 = 6 comp.windows.x || 113번째 단어 drive/ 312번째 단어 please/ 36

In [71]:
# 문서별로 분류된 topic 코드 확인
def check_topic(x, y):
    print("문서 %d의 topic = %s" %(x, news_data.target_names[news_data.target[x]]))
    print("문서 %d의 topic = %s" %(y, news_data.target_names[news_data.target[y]]))

check_topic(1, 6)
check_topic(0, 2)

문서 1의 topic = alt.atheism
문서 6의 topic = comp.sys.mac.hardware
문서 0의 topic = talk.politics.mideast
문서 2의 topic = talk.politics.mideast
